In [27]:
import sys
from pathlib import Path
import pandas as pd 
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import SVC
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV

sys.path.append(str(Path("../..").resolve()))
from src.constants import model_data_dir
from src.utils import use_target, BlockingTimeSeriesSplit


In [35]:
svc = SVC()
C_range = np.logspace(-1, 1, 3)
gamma_range = np.logspace(-1, 1, 3)
kernel_range = ["rbf", "poly"]

# Define the search space
param_grid = { 
    # Regularization parameter.
    "C": C_range,
    # Kernel type
    "kernel": kernel_range, 
    # Gamma is the Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.
    "gamma": gamma_range.tolist()+['scale', 'auto']
}

folds = BlockingTimeSeriesSplit(n_splits=5)

scoring = 'accuracy'

In [6]:
df_train = pd.read_csv(model_data_dir / "train_classification.csv")
df_test = pd.read_csv(model_data_dir / "test_classification.csv")
test_dates = pd.to_datetime(df_test["date"])
df_train = df_train.drop(columns=["date"])
df_test = df_test.drop("date", axis=1)

In [7]:
df_train

,vix,v2x,move,gold_vol,jpm_global_fx_vol,jpm_g7_fx_vol,jpm_em_fx_vol,eurusd_1m_vol,usdjpy_1m_vol,usd_libor_swaption_vol,...,eurusd_risk_reversal_rolling_40,usdjpy_risk_reversal_rolling_40,eurjpy_risk_reversal_rolling_40,month,year,day,weekday,sc_1d_fwd_rel_d,mom_1d_fwd_rel_d,value_1d_fwd_rel_d
0,-0.0226,-0.2926,0.0748,-0.5463,0.1855,0.3734,-0.7876,0.4775,0.2698,-1.2457,...,-0.2138,-0.1097,-0.1090,7,2000,26,2,0,0,1
1,-0.0305,-0.2322,0.0164,4.7611,0.1855,0.3766,-0.7994,0.3823,0.2698,-1.2551,...,-0.2027,-0.1139,-0.0988,7,2000,27,3,1,0,1
2,0.1082,-0.1034,0.0001,-0.5638,0.1960,0.3832,-0.7700,0.4299,0.2859,-1.2551,...,-0.1929,-0.1144,-0.0896,7,2000,28,4,1,1,1
3,0.0970,-0.1331,0.0318,-0.5638,0.2168,0.4061,-0.7847,0.4934,0.3020,-1.2504,...,-0.1897,-0.1105,-0.0814,7,2000,31,0,0,0,1
4,0.0758,-0.1123,0.0411,0.2705,0.2064,0.3963,-0.7847,0.4299,0.2859,-1.2457,...,-0.1868,-0.1069,-0.0768,8,2000,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3374,-0.8267,4.3768,-0.5865,-0.0682,-0.5133,-0.4604,-0.3578,-0.9052,-0.2861,-0.0529,...,-0.3320,-0.2409,-0.2535,12,2013,24,1,0,1,1
3375,-0.8434,-1.4408,-0.6189,-0.0850,-0.4612,-0.4016,-0.3519,-0.8901,-0.1405,-0.0037,...,-0.3342,-0.2398,-0.2535,12,2013,26,3,0,0,1
3376,-0.8289,-0.7063,-0.6488,-0.1348,-0.4160,-0.3656,-0.2900,-0.7480,-0.0415,0.0248,...,-0.3347,-0.2376,-0.2535,12,2013,27,4,0,0,1
3377,-0.7059,-0.6280,-0.5449,-0.1214,-0.4090,-0.3558,-0.2930,-0.5353,-0.0962,0.0031,...,-0.3346,-0.2351,-0.2538,12,2013,30,0,0,1,1


## Size

In [21]:
y_train_size, x_train_size = use_target(df_train, "sc_1d_fwd_rel_d", "classification")
y_test_size, x_test_size = use_target(df_test, "sc_1d_fwd_rel_d", "classification")

In [22]:
x_train_size

,vix,v2x,move,gold_vol,jpm_global_fx_vol,jpm_g7_fx_vol,jpm_em_fx_vol,eurusd_1m_vol,usdjpy_1m_vol,usd_libor_swaption_vol,...,usd_libor_swaption_vol_rolling_40,s&p_500_vrp_rolling_40,s&p_500_skew_rolling_40,eurusd_risk_reversal_rolling_40,usdjpy_risk_reversal_rolling_40,eurjpy_risk_reversal_rolling_40,month,year,day,weekday
0,-0.0226,-0.2926,0.0748,-0.5463,0.1855,0.3734,-0.7876,0.4775,0.2698,-1.2457,...,-0.6174,-1.0756,-0.9061,-0.2138,-0.1097,-0.1090,7,2000,26,2
1,-0.0305,-0.2322,0.0164,4.7611,0.1855,0.3766,-0.7994,0.3823,0.2698,-1.2551,...,-0.6174,-1.0756,-0.9010,-0.2027,-0.1139,-0.0988,7,2000,27,3
2,0.1082,-0.1034,0.0001,-0.5638,0.1960,0.3832,-0.7700,0.4299,0.2859,-1.2551,...,-0.6168,-1.0756,-0.9067,-0.1929,-0.1144,-0.0896,7,2000,28,4
3,0.0970,-0.1331,0.0318,-0.5638,0.2168,0.4061,-0.7847,0.4934,0.3020,-1.2504,...,-0.6161,-1.0756,-0.9030,-0.1897,-0.1105,-0.0814,7,2000,31,0
4,0.0758,-0.1123,0.0411,0.2705,0.2064,0.3963,-0.7847,0.4299,0.2859,-1.2457,...,-0.6161,-1.0756,-0.8965,-0.1868,-0.1069,-0.0768,8,2000,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3374,-0.8267,4.3768,-0.5865,-0.0682,-0.5133,-0.4604,-0.3578,-0.9052,-0.2861,-0.0529,...,-0.3442,-0.5041,0.4184,-0.3320,-0.2409,-0.2535,12,2013,24,1
3375,-0.8434,-1.4408,-0.6189,-0.0850,-0.4612,-0.4016,-0.3519,-0.8901,-0.1405,-0.0037,...,-0.3360,-0.4979,0.4465,-0.3342,-0.2398,-0.2535,12,2013,26,3
3376,-0.8289,-0.7063,-0.6488,-0.1348,-0.4160,-0.3656,-0.2900,-0.7480,-0.0415,0.0248,...,-0.3299,-0.4907,0.4732,-0.3347,-0.2376,-0.2535,12,2013,27,4
3377,-0.7059,-0.6280,-0.5449,-0.1214,-0.4090,-0.3558,-0.2930,-0.5353,-0.0962,0.0031,...,-0.3184,-0.4901,0.4828,-0.3346,-0.2351,-0.2538,12,2013,30,0


In [36]:
# Define grid search
grid_search = HalvingGridSearchCV(
    estimator=svc, 
    param_grid=param_grid, 
    scoring=scoring, 
    n_jobs=-1, 
    cv=folds, 
    resource='n_samples', 
    verbose=0
)

grid_result = grid_search.fit(x_train_size, y_train_size)


In [41]:
grid_result.best_estimator_.score(x_test_size, y_test_size)

0.48886532343584305

## Momentum

## Value